In [7]:
from elasticsearch import Elasticsearch
import pandas as pd
from pandas.io.json import json_normalize
import datetime

In [13]:
from pprint import pprint

In [58]:
def to_datetime_utc(s):
    dt = datetime.datetime.utcfromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d %H")

In [61]:
citys={'中山市','茂名市','佛山市','阳江市'}


In [65]:
dsl_m_now = {'query': {'bool': {'must': [{'range': {'paytime': {'gte': '2019-05-27 15:00:00', 'lte': '2019-05-27 15:23:59', 'format': 'yyyy-MM-dd HH:mm:ss', 'time_zone': '+00:00'}}}, {'terms': {'group_city': [city]}}, {'terms': {'order_type': ['vip_scan', 'deal']}}, {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}, {'range': {'card_discount': {'gt': 0}}}]}}, 'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [{'日期': {'date_histogram': {'field': 'paytime', 'interval': '1h', 'time_zone': '+00:00'}}}, {'group_city': {'terms': {'field': 'group_city'}}}]}, 'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}}}}
es_m_now = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res_m_now = es_m.search(index="es_order_table", doc_type="order", body=dsl_m_now,
            _source=True, explain=False, track_scores=False, track_total_hits=False)
df_m_now  = json_normalize(res_m_now['aggregations']['my_buckets']['buckets'])
df_m_now ['key.日期'] = df_m_now ['key.日期'].apply(to_datetime_utc)

In [70]:
for city in citys:
    dsl_m_now = {'query': {'bool': {'must': [{'range': {'paytime': {'gte': '2019-05-27', 'lte': '2019-05-27', 'format': 'yyyy-MM-dd', 'time_zone': '+00:00'}}}, {'terms': {'group_city': [city]}}, {'terms': {'order_type': ['vip_scan', 'deal']}}, {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}, {'range': {'card_discount': {'gt': 0}}}]}}, 'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [{'日期': {'date_histogram': {'field': 'paytime', 'interval': '1h', 'time_zone': '+00:00'}}}, {'group_city': {'terms': {'field': 'group_city'}}}]}, 'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}}}}
    es_m_now = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_m_now = es_m.search(index="es_order_table", doc_type="order", body=dsl_m_now,
                _source=True, explain=False, track_scores=False, track_total_hits=False)
    df_m_now  = json_normalize(res_m_now['aggregations']['my_buckets']['buckets'])
    df_m_now ['key.日期'] = df_m_now ['key.日期'].apply(to_datetime_utc)
    dsl_m_yes={'query': {'bool': {'must': [{'range': {'paytime': {'gte': '2019-05-26', 'lte': '2019-05-26', 'format': 'yyyy-MM-dd', 'time_zone': '+00:00'}}}, {'terms': {'group_city': ['茂名市']}}, {'terms': {'order_type': ['vip_scan', 'deal']}}, {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}, {'range': {'card_discount': {'gt': 0}}}]}}, 'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [{'日期': {'date_histogram': {'field': 'paytime', 'interval': '1h', 'time_zone': '+00:00'}}}, {'group_city': {'terms': {'field': 'group_city'}}}]}, 'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}}}}
    es_m_yes = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_m_yes = es_m_yes.search(index="es_order_table", doc_type="order", body=dsl_m_yes,
                _source=True, explain=False, track_scores=False, track_total_hits=False)
    df_m_yes  = json_normalize(res_m_yes['aggregations']['my_buckets']['buckets'])
    df_m_yes ['key.日期'] = df_m_yes ['key.日期'].apply(to_datetime_utc)
    if (df_m_now ['折前GMV.value'][len(df_m_now)-2]-df_m_yes ['折前GMV.value'][len(df_m_now)-2])/df_m_yes ['折前GMV.value'][len(df_m_now)-2] >0.2:#与昨天对比警报
        print(city+'环比'+'异常')
    else:
        print(city+'环比'+'正常')
    dsl_m_sev = {'query': {'bool': {'must': [{'range': {'paytime': {'gte': '2019-05-26', 'lte': '2019-05-26', 'format': 'yyyy-MM-dd', 'time_zone': '+00:00'}}}, {'terms': {'group_city': ['茂名市']}}, {'terms': {'order_type': ['vip_scan', 'deal']}}, {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}, {'range': {'card_discount': {'gt': 0}}}]}}, 'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [{'日期': {'date_histogram': {'field': 'paytime', 'interval': '1h', 'time_zone': '+00:00'}}}, {'group_city': {'terms': {'field': 'group_city'}}}]}, 'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}}}}
    es_m_sev = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_m_sev = es_m_sev.search(index="es_order_table", doc_type="order", body=dsl_m_sev,
                _source=True, explain=False, track_scores=False, track_total_hits=False)
    df_m_sev  = json_normalize(res_m_sev['aggregations']['my_buckets']['buckets'])
    df_m_sev ['key.日期'] = df_m_sev ['key.日期'].apply(to_datetime_utc)
    if (df_m_now ['折前GMV.value'][len(df_m_now)-2]-df_m_sev ['折前GMV.value'][len(df_m_now)-2])/df_m_sev ['折前GMV.value'][len(df_m_now)-2] >0.15:#与七天前对比警报
        print(city+'同比'+'异常')
    else:
        print(city+'同比'+'正常')
        
    
    

茂名市环比异常
茂名市同比异常
佛山市环比正常
佛山市同比正常
中山市环比正常
中山市同比正常
阳江市环比正常
阳江市同比正常
